# 🎬 Multi-Modal Video Summarizer

A comprehensive video summarization system that analyzes both **audio** and **visual** content to generate detailed summaries.

## Features
- ✅ **Audio Transcription**: Extracts and transcribes speech using Whisper
- ✅ **Visual Analysis**: Captions frames using BLIP vision-language model
- ✅ **OCR Text Extraction**: Extracts on-screen text from video frames
- ✅ **Scene Understanding**: Analyzes visual context and temporal changes
- ✅ **No-Audio Support**: Works even when videos have no audio
- ✅ **100% Free**: Uses only open-source models (no paid APIs)
- ✅ **Colab Optimized**: Designed for Google Colab free tier (T4 GPU)

## Industry-Standard Technologies
- **Whisper** (OpenAI): State-of-the-art speech recognition
- **BLIP** (Salesforce): Vision-language understanding
- **EasyOCR**: Text extraction from images
- **T5/BART**: Text summarization models
- **PyTorch**: Deep learning framework


## 📦 Installation & Setup


In [ ]:
# Install required packages
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers accelerate
!pip install -q opencv-python pillow
!pip install -q librosa soundfile
!pip install -q easyocr
!pip install -q moviepy
!pip install -q sentencepiece protobuf

print("✅ All packages installed successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 12.3 MB/s eta 0:00:00
✅ All packages installed successfully!


## 🔧 Import Libraries


In [ ]:
import cv2
import torch
import torch.nn as nn
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    T5Tokenizer,
    T5ForConditionalGeneration,
    pipeline
)
import tempfile
import os
import numpy as np
import librosa
import subprocess
import easyocr
from PIL import Image
from moviepy.editor import VideoFileClip
from typing import List, Optional, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



✅ Libraries imported successfully!


## 🚀 Initialize Models


In [3]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print("\n🔄 Loading models... This may take a few minutes...")


Using device: cuda
GPU: Tesla T4
GPU Memory: 15.83 GB

🔄 Loading models... This may take a few minutes...


In [ ]:
# Load BLIP model for visual captioning (lightweight base model for Colab)
print("Loading BLIP model for visual analysis...")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = blip_model.to(device)
blip_model.eval()
print("✅ BLIP model loaded")

# Load Whisper model for audio transcription (base model for Colab T4)
print("Loading Whisper model for audio transcription...")
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-base")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
whisper_model = whisper_model.to(device)
whisper_model.eval()
print("✅ Whisper model loaded")

# Load T5 model for summarization (small model for Colab)
print("Loading T5 model for text summarization...")
summarizer_tokenizer = T5Tokenizer.from_pretrained("t5-small")
summarizer_model = T5ForConditionalGeneration.from_pretrained("t5-small")
summarizer_model = summarizer_model.to(device)
summarizer_model.eval()
print("✅ T5 summarizer model loaded")

# Initialize EasyOCR (lightweight, free OCR)
print("Initializing EasyOCR...")
ocr_reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
print("✅ EasyOCR initialized")

print("\n🎉 All models loaded successfully!")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading BLIP model for visual analysis...


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

✅ Whisper model loaded
Initializing EasyOCR...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ EasyOCR initialized

🎉 All models loaded successfully!

📝 Note: LLM API keys need to be configured for human-readable summaries


## 🎥 Video Processing Functions


In [5]:
def extract_keyframes(video_path: str, num_frames: int = 10) -> List[np.ndarray]:
    """
    Extract evenly spaced keyframes from video.

    Args:
        video_path: Path to video file
        num_frames: Number of frames to extract

    Returns:
        List of frames as numpy arrays (RGB format)
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps if fps > 0 else 0

    print(f"Video info: {total_frames} frames, {fps:.2f} FPS, {duration:.2f}s duration")

    # Calculate frame indices to extract
    if total_frames < num_frames:
        frame_indices = list(range(total_frames))
    else:
        frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            # Convert BGR to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

    cap.release()
    print(f"✅ Extracted {len(frames)} frames")
    return frames


In [6]:
def extract_audio_from_video(video_path: str) -> Tuple[Optional[np.ndarray], Optional[int]]:
    """
    Extract audio from video file.

    Args:
        video_path: Path to video file

    Returns:
        Tuple of (audio_data, sample_rate) or (None, None) if no audio
    """
    try:
        video_clip = VideoFileClip(video_path)

        if video_clip.audio is None:
            video_clip.close()
            print("⚠️ No audio track found in video")
            return None, None

        # Save audio as temporary wav file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_audio:
            audio_path = tmp_audio.name
            video_clip.audio.write_audiofile(audio_path, verbose=False, logger=None)

        # Load audio with librosa (Whisper expects 16kHz)
        audio_data, sample_rate = librosa.load(audio_path, sr=16000)

        # Cleanup
        video_clip.close()
        os.remove(audio_path)

        print(f"✅ Audio extracted: {len(audio_data)} samples, {sample_rate} Hz")
        return audio_data, sample_rate

    except Exception as e:
        print(f"⚠️ Error extracting audio: {e}")
        return None, None


## 🎤 Audio Processing


In [7]:
def transcribe_audio(audio_data: np.ndarray, sample_rate: int) -> str:
    """
    Transcribe audio using Whisper model.

    Args:
        audio_data: Audio waveform array
        sample_rate: Sample rate of audio

    Returns:
        Transcribed text
    """
    if audio_data is None or len(audio_data) == 0:
        return "No audio content detected."

    try:
        # Prepare audio for Whisper
        inputs = whisper_processor(audio_data, sampling_rate=sample_rate, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate transcription
        with torch.no_grad():
            predicted_ids = whisper_model.generate(
                inputs["input_features"],
                max_length=448,
                num_beams=5,
                language="en"
            )

        # Decode transcription
        transcription = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

        return transcription.strip() if transcription.strip() else "No speech detected in audio."

    except Exception as e:
        print(f"⚠️ Error in transcription: {e}")
        return "Audio transcription failed."


## 👁️ Visual Processing


In [8]:
def caption_frame(frame: np.ndarray) -> str:
    """
    Generate caption for a single frame using BLIP.

    Args:
        frame: Frame as numpy array (RGB)

    Returns:
        Caption describing the frame
    """
    try:
        # Convert numpy array to PIL Image
        image = Image.fromarray(frame)

        # Process with BLIP
        inputs = blip_processor(image, return_tensors="pt").to(device)

        with torch.no_grad():
            out = blip_model.generate(
                **inputs,
                max_length=50,
                num_beams=3,
                do_sample=False
            )

        caption = blip_processor.decode(out[0], skip_special_tokens=True)
        return caption

    except Exception as e:
        print(f"⚠️ Error captioning frame: {e}")
        return "Unable to describe this frame."


In [9]:
def extract_text_from_frame(frame: np.ndarray) -> List[str]:
    """
    Extract text from frame using OCR.

    Args:
        frame: Frame as numpy array (RGB)

    Returns:
        List of extracted text strings
    """
    try:
        # EasyOCR expects numpy array
        results = ocr_reader.readtext(frame)

        # Filter by confidence threshold
        texts = [text[1] for text in results if text[2] > 0.5]
        return texts

    except Exception as e:
        print(f"⚠️ Error in OCR: {e}")
        return []


In [10]:
def analyze_visual_content(frames: List[np.ndarray]) -> Dict[str, any]:
    """
    Analyze all frames to extract visual information.

    Args:
        frames: List of video frames

    Returns:
        Dictionary containing visual analysis results
    """
    print(f"\n🔍 Analyzing {len(frames)} frames...")

    frame_captions = []
    all_ocr_texts = []

    for i, frame in enumerate(frames):
        print(f"Processing frame {i+1}/{len(frames)}...", end="\r")

        # Generate caption
        caption = caption_frame(frame)
        frame_captions.append(f"Frame {i+1}: {caption}")

        # Extract text
        ocr_texts = extract_text_from_frame(frame)
        if ocr_texts:
            all_ocr_texts.extend(ocr_texts)
            frame_captions[-1] += f" | On-screen text: {', '.join(ocr_texts)}"

    print("\n✅ Visual analysis complete")

    return {
        "frame_captions": frame_captions,
        "ocr_texts": all_ocr_texts,
        "visual_summary": "\n".join(frame_captions)
    }


## 📝 Text Summarization


In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

def summarize_text(text: str, max_length: int = 150, min_length: int = 30) -> str:
    """
    Summarize text using T5 model.

    Args:
        text: Text to summarize
        max_length: Maximum length of summary
        min_length: Minimum length of summary

    Returns:
        Summarized text
    """
    if not text or len(text.strip()) < 50:
        return text

    try:
        # T5 requires "summarize:" prefix
        input_text = f"summarize: {text}"

        # Tokenize
        inputs = summarizer_tokenizer(
            input_text,
            max_length=512,
            truncation=True,
            return_tensors="pt"
        ).to(device)

        # Generate summary
        with torch.no_grad():
            summary_ids = summarizer_model.generate(
                inputs["input_ids"],
                max_length=max_length,
                min_length=min_length,
                num_beams=4,
                length_penalty=2.0,
                early_stopping=True
            )

        summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    except Exception as e:
        print(f"⚠️ Error in summarization: {e}")
        return text  # Return original if summarization fails


## 🤖 LLM Configuration for Human-Readable Summaries

Configure your LLM API keys here. You can use either Groq or Gemini (or both).


In [ ]:
# ============================================
# 🔑 API KEY CONFIGURATION
# ============================================
# Add your API keys here:

# Option 1: Groq API Key (for LLaMA models)
GROQ_API_KEY = ""  # 👈 ADD YOUR GROQ API KEY HERE
# Get your key from: https://console.groq.com/

# Option 2: Google Gemini API Key
GEMINI_API_KEY = ""  # 👈 ADD YOUR GEMINI API KEY HERE
# Get your key from: https://makersuite.google.com/app/apikey

# Preferred LLM (options: "groq", "gemini", or "both")
PREFERRED_LLM = "groq"  # Change to "gemini" if you prefer Gemini

# ============================================
# Initialize LLM Clients
# ============================================

if GROQ_API_KEY and GROQ_AVAILABLE:
    try:
        groq_client = Groq(api_key=GROQ_API_KEY)
        print("✅ Groq client initialized")
    except Exception as e:
        print(f"⚠️ Error initializing Groq: {e}")
        groq_client = None
else:
    if not GROQ_API_KEY:
        print("⚠️ Groq API key not set")
    groq_client = None

if GEMINI_API_KEY and GEMINI_AVAILABLE:
    try:
        genai.configure(api_key=GEMINI_API_KEY)
        gemini_model = genai.GenerativeModel('gemini-pro')
        print("✅ Gemini client initialized")
    except Exception as e:
        print(f"⚠️ Error initializing Gemini: {e}")
        gemini_model = None
else:
    if not GEMINI_API_KEY:
        print("⚠️ Gemini API key not set")
    gemini_model = None

if not groq_client and not gemini_model:
    print("\n⚠️ WARNING: No LLM configured. Human-readable summaries will use fallback method.")
    print("Please set at least one API key above.")

✅ Groq client initialized
✅ Gemini client initialized


In [31]:
# Summary style options
SUMMARY_STYLES = {
    "default": {
        "name": "Default Summary",
        "description": "Standard comprehensive summary"
    },
    "professional": {
        "name": "Professional Summary",
        "description": "Formal, business-oriented summary suitable for reports and presentations"
    },
    "commercial": {
        "name": "Commercial Advertisement Summary",
        "description": "Marketing-focused summary highlighting key selling points and benefits"
    },
    "educational": {
        "name": "Educational Summary",
        "description": "Academic-style summary focusing on learning objectives and key concepts"
    },
    "casual": {
        "name": "Casual Summary",
        "description": "Relaxed, conversational summary in everyday language"
    },
    "technical": {
        "name": "Technical Summary",
        "description": "Detailed technical summary with specific terminology and specifications"
    }
}

def get_summary_prompt(style: str, full_context: str) -> str:
    """
    Get the appropriate prompt based on summary style.
    """
    style_prompts = {
        "default": f"""You are an expert video summarizer. Based on the following information extracted from a video, write a comprehensive, human-readable summary in paragraph form (2-4 paragraphs).

The summary should:
1. Be written like a professional article or essay summary
2. Flow naturally and be easy to read
3. Combine information from audio, visual content, and on-screen text seamlessly
4. Highlight the main topics, themes, and key information
5. Be informative and engaging
6. If there's no audio, focus on the visual content and on-screen text
7. Write in complete sentences with proper grammar

VIDEO INFORMATION:
{full_context}

Write a comprehensive summary:""",

        "professional": f"""You are a professional business analyst. Based on the following video information, write a formal, professional summary suitable for business reports or presentations (2-3 paragraphs).

The summary should:
1. Use formal, professional language
2. Focus on key business insights, objectives, and outcomes
3. Be concise and structured
4. Highlight actionable items or important decisions
5. Maintain a professional tone throughout
6. Be suitable for executive summaries or meeting notes

VIDEO INFORMATION:
{full_context}

Write a professional business summary:""",

        "commercial": f"""You are a marketing copywriter. Based on the following video information, write an engaging commercial/advertisement summary that highlights key selling points (2-3 paragraphs).

The summary should:
1. Be persuasive and engaging
2. Highlight benefits, features, and value propositions
3. Use compelling language that captures attention
4. Focus on what makes the content/product/service appealing
5. Include call-to-action elements if appropriate
6. Be suitable for marketing materials or promotional content

VIDEO INFORMATION:
{full_context}

Write a commercial advertisement summary:""",

        "educational": f"""You are an educational content specialist. Based on the following video information, write an academic-style summary focusing on learning outcomes (2-3 paragraphs).

The summary should:
1. Focus on key concepts, learning objectives, and educational value
2. Use clear, instructional language
3. Highlight important facts, theories, or information presented
4. Be suitable for study notes or educational materials
5. Organize information logically for learning purposes
6. Emphasize what viewers can learn from the content

VIDEO INFORMATION:
{full_context}

Write an educational summary:""",

        "casual": f"""You are a friendly content creator. Based on the following video information, write a casual, conversational summary in everyday language (2-3 paragraphs).

The summary should:
1. Use relaxed, conversational tone
2. Be easy to read and understand
3. Feel like a friend explaining the video to you
4. Use natural, everyday language
5. Be engaging and relatable
6. Avoid overly formal or technical terms

VIDEO INFORMATION:
{full_context}

Write a casual, friendly summary:""",

        "technical": f"""You are a technical documentation specialist. Based on the following video information, write a detailed technical summary with specific terminology (2-4 paragraphs).

The summary should:
1. Use precise technical language and terminology
2. Include specific details, specifications, and technical information
3. Be comprehensive and detailed
4. Focus on technical aspects, processes, and methodologies
5. Be suitable for technical documentation or engineering notes
6. Maintain accuracy and precision

VIDEO INFORMATION:
{full_context}

Write a technical summary:"""
    }

    return style_prompts.get(style, style_prompts["default"])

def generate_human_readable_summary(
    audio_transcription: str,
    visual_analysis: Dict,
    ocr_texts: List[str],
    llm_provider: str = "groq",
    summary_style: str = "default"
) -> str:
    """
    Generate a human-readable paragraph summary using LLM.
    This creates a cohesive summary like you would write for an essay or article.

    Args:
        audio_transcription: Transcribed audio content
        visual_analysis: Dictionary with frame captions and visual summary
        ocr_texts: List of extracted text from frames
        llm_provider: Which LLM to use ("groq" or "gemini")
        summary_style: Style of summary ("default", "professional", "commercial", "educational", "casual", "technical")

    Returns:
        Human-readable paragraph summary
    """

    # Validate summary style
    if summary_style not in SUMMARY_STYLES:
        print(f"⚠️ Invalid summary style '{summary_style}', using 'default'")
        summary_style = "default"

    print(f"📝 Generating {SUMMARY_STYLES[summary_style]['name']}...")

    # Build comprehensive context
    context_parts = []

    # Audio content
    if audio_transcription and "No audio" not in audio_transcription:
        context_parts.append(f"AUDIO TRANSCRIPTION:\n{audio_transcription}")

    # Visual content - extract key scenes
    visual_scenes = []
    for caption in visual_analysis.get("frame_captions", []):
        # Extract just the description part
        if ": " in caption:
            scene_desc = caption.split(": ", 1)[-1]
            # Remove OCR text part if present
            if " | On-screen text:" in scene_desc:
                scene_desc = scene_desc.split(" | On-screen text:")[0]
            visual_scenes.append(scene_desc.strip())

    if visual_scenes:
        context_parts.append(f"VISUAL CONTENT (Key Scenes):\n" + "\n".join(visual_scenes[:10]))

    # OCR text
    if ocr_texts:
        unique_texts = list(set(ocr_texts))[:10]
        context_parts.append(f"ON-SCREEN TEXT:\n" + ", ".join(unique_texts))

    full_context = "\n\n".join(context_parts)

    # Get style-specific prompt
    prompt = get_summary_prompt(summary_style, full_context)

    # Try to use LLM
    summary = None

    # Try Groq first (if requested and available)
    if llm_provider == "groq" and groq_client:
        try:
            response = groq_client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {"role": "system", "content": f"You are an expert video summarizer. Write clear, comprehensive summaries in paragraph form according to the {SUMMARY_STYLES[summary_style]['name']} style."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=250
            )
            summary = response.choices[0].message.content.strip()
            print("✅ Summary generated using Groq (LLaMA 3.3)")
        except Exception as e:
            print(f"⚠️ Groq error: {e}")
            summary = None

    # Try Gemini if Groq failed or if Gemini is preferred
    if (not summary or llm_provider == "gemini") and gemini_model:
        try:
            response = gemini_model.generate_content(prompt)
            summary = response.text.strip()
            print("✅ Summary generated using Gemini")
        except Exception as e:
            print(f"⚠️ Gemini error: {e}")
            if not summary:
                summary = None

    # Fallback: Create structured summary if LLM fails
    if not summary:
        print("⚠️ Using fallback summary method (no LLM available)")
        summary = create_fallback_summary(audio_transcription, visual_analysis, ocr_texts)

    return summary


def create_fallback_summary(
    audio_transcription: str,
    visual_analysis: Dict,
    ocr_texts: List[str]
) -> str:
    """
    Create a readable summary without LLM (fallback method).
    """
    paragraphs = []

    # Introduction paragraph
    intro = "This video presents content covering multiple aspects. "

    # Audio paragraph
    if audio_transcription and "No audio" not in audio_transcription:
        audio_summary = audio_transcription[:300] + "..." if len(audio_transcription) > 300 else audio_transcription
        paragraphs.append(f"The audio narration discusses: {audio_summary}")

    # Visual paragraph
    if visual_analysis.get("frame_captions"):
        key_scenes = []
        for caption in visual_analysis["frame_captions"][:5]:
            if ": " in caption:
                scene = caption.split(": ", 1)[-1]
                if " | On-screen text:" in scene:
                    scene = scene.split(" | On-screen text:")[0]
                key_scenes.append(scene.strip())

        if key_scenes:
            visual_desc = ", ".join(key_scenes[:3])
            paragraphs.append(f"Visually, the video shows: {visual_desc}.")

    # OCR paragraph
    if ocr_texts:
        unique_texts = list(set(ocr_texts))[:5]
        paragraphs.append(f"Notable on-screen text includes: {', '.join(unique_texts)}.")

    # Combine into readable format
    if paragraphs:
        return intro + " ".join(paragraphs)
    else:
        return "The video contains visual content that has been analyzed, but no detailed summary could be generated without LLM support."


## 🎯 Main Processing Pipeline


In [16]:
def create_structured_summary(results: Dict, visual_analysis: Dict) -> str:
    """
    Create a structured summary when automatic summarization is insufficient.
    """
    summary_parts = ["🎬 VIDEO SUMMARY\n" + "=" * 50]

    # Audio section
    if results["audio_transcription"] and "No audio" not in results["audio_transcription"]:
        summary_parts.append("\n📢 AUDIO CONTENT:")
        summary_parts.append(results["audio_transcription"])

    # Visual section
    summary_parts.append("\n👁️ VISUAL CONTENT:")

    # Extract key themes from frame captions
    all_captions = " ".join(visual_analysis["frame_captions"])

    # Identify common themes (simple keyword extraction)
    key_scenes = []
    for caption in visual_analysis["frame_captions"][:5]:  # Top 5 frames
        key_scenes.append(caption.split(": ", 1)[-1] if ": " in caption else caption)

    summary_parts.append("\n".join(key_scenes[:3]))  # Top 3 scenes

    # OCR section
    if visual_analysis["ocr_texts"]:
        unique_texts = list(set(visual_analysis["ocr_texts"]))[:5]  # Top 5 unique texts
        summary_parts.append("\n📝 ON-SCREEN TEXT:")
        summary_parts.append(", ".join(unique_texts))

    # Overall description
    summary_parts.append("\n\n📋 OVERALL DESCRIPTION:")

    if results["audio_transcription"] and "No audio" not in results["audio_transcription"]:
        summary_parts.append(
            f"This video contains audio narration discussing: {results['audio_transcription'][:200]}... "
            f"Visually, the video shows: {key_scenes[0] if key_scenes else 'various scenes'}."
        )
    else:
        summary_parts.append(
            f"This is a visual-only video showing: {', '.join(key_scenes[:3]) if key_scenes else 'various scenes'}. "
            f"The video contains {len(visual_analysis['frame_captions'])} distinct scenes analyzed."
        )

    return "\n".join(summary_parts)


In [17]:
def process_video(video_path: str, num_frames: int = 10, use_llm: bool = True, summary_style: str = "default") -> Dict[str, str]:
    """
    Main function to process video and generate comprehensive, human-readable summary.

    Args:
        video_path: Path to video file
        num_frames: Number of frames to extract for analysis
        use_llm: Whether to use LLM for human-readable summary (default: True)
        summary_style: Style of summary ("default", "professional", "commercial", "educational", "casual", "technical")

    Returns:
        Dictionary containing all analysis results and final human-readable summary
    """
    print("=" * 60)
    print("🎬 VIDEO SUMMARIZATION PIPELINE")
    print("=" * 60)

    results = {
        "audio_transcription": "",
        "visual_analysis": {},
        "final_summary": "",
        "status": "processing",
        "summary_style": summary_style
    }

    # Step 1: Extract keyframes
    print("\n📹 Step 1: Extracting keyframes...")
    frames = extract_keyframes(video_path, num_frames)

    if not frames:
        return {"error": "Could not extract frames from video", "status": "error"}

    # Step 2: Extract and transcribe audio
    print("\n🎤 Step 2: Processing audio...")
    audio_data, sample_rate = extract_audio_from_video(video_path)

    if audio_data is not None:
        audio_transcription = transcribe_audio(audio_data, sample_rate)
        results["audio_transcription"] = audio_transcription
        print(f"\n📝 Audio Transcription:\n{audio_transcription[:200]}...")
    else:
        results["audio_transcription"] = "No audio track found in video."
        print("\n⚠️ No audio detected - proceeding with visual-only analysis")

    # Step 3: Analyze visual content
    print("\n👁️ Step 3: Analyzing visual content...")
    visual_analysis = analyze_visual_content(frames)
    results["visual_analysis"] = visual_analysis

    # Step 4: Generate human-readable summary using LLM
    print(f"\n📝 Step 4: Generating {SUMMARY_STYLES.get(summary_style, SUMMARY_STYLES['default'])['name']}...")

    if use_llm and (groq_client or gemini_model):
        # Use LLM for high-quality summary
        llm_provider = PREFERRED_LLM if (PREFERRED_LLM == "groq" and groq_client) or (PREFERRED_LLM == "gemini" and gemini_model) else ("groq" if groq_client else "gemini")

        final_summary = generate_human_readable_summary(
            results["audio_transcription"],
            visual_analysis,
            visual_analysis.get("ocr_texts", []),
            llm_provider=llm_provider,
            summary_style=summary_style
        )
    else:
        # Fallback method
        print("⚠️ Using fallback summary (no LLM configured)")
        final_summary = create_fallback_summary(
            results["audio_transcription"],
            visual_analysis,
            visual_analysis.get("ocr_texts", [])
        )

    results["final_summary"] = final_summary
    results["status"] = "completed"

    print("\n" + "=" * 60)
    print("✅ PROCESSING COMPLETE")
    print("=" * 60)

    return results


## 🎨 Summary Style Selection

Choose your preferred summary style:


In [18]:
# Display available summary styles
print("📋 Available Summary Styles:")
print("=" * 60)
for style_key, style_info in SUMMARY_STYLES.items():
    print(f"\n{style_key.upper()}: {style_info['name']}")
    print(f"  {style_info['description']}")

# Set your preferred style here (or pass it to process_video function)
PREFERRED_SUMMARY_STYLE = "default"  # Options: "default", "professional", "commercial", "educational", "casual", "technical"

print(f"\n✅ Current preferred style: {PREFERRED_SUMMARY_STYLE}")
print(f"   {SUMMARY_STYLES[PREFERRED_SUMMARY_STYLE]['name']}")


📋 Available Summary Styles:

DEFAULT: Default Summary
  Standard comprehensive summary

PROFESSIONAL: Professional Summary
  Formal, business-oriented summary suitable for reports and presentations

COMMERCIAL: Commercial Advertisement Summary
  Marketing-focused summary highlighting key selling points and benefits

EDUCATIONAL: Educational Summary
  Academic-style summary focusing on learning objectives and key concepts

CASUAL: Casual Summary
  Relaxed, conversational summary in everyday language

TECHNICAL: Technical Summary
  Detailed technical summary with specific terminology and specifications

✅ Current preferred style: default
   Default Summary


## 🎬 Usage Example


In [19]:
# Upload your video file
# Option 1: For Google Colab - use files.upload()
# Option 2: For local Jupyter - provide path directly

try:
    # Try Colab first
    from google.colab import files
    print("Running on Google Colab")
    print("Please upload your video file:")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0] if uploaded else None
    if video_path:
        print(f"\n✅ Video uploaded: {video_path}")
    else:
        print("\n⚠️ No video uploaded.")
        video_path = None
except ImportError:
    # Running locally
    print("Running locally - please provide video path:")
    # Uncomment and set your video path:
    # video_path = "/path/to/your/video.mp4"
    video_path = None
    if video_path:
        print(f"\n✅ Using video: {video_path}")
    else:
        print("\n⚠️ Please set video_path variable with your video file path")


Running on Google Colab
Please upload your video file:


Saving vedio1.mp4 to vedio1.mp4

✅ Video uploaded: vedio1.mp4


In [21]:
# Process the video
video_path = True
if video_path:
    results = process_video('/content/vedio1.mp4', num_frames=30)

    # Display results
    print("\n" + "=" * 60)
    print("📊 FINAL SUMMARY")
    print("=" * 60)
    print(results["final_summary"])

    # Save results to file
    with open("video_summary.txt", "w") as f:
        f.write("VIDEO SUMMARIZATION RESULTS\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Audio Transcription:\n{results['audio_transcription']}\n\n")
        f.write(f"Visual Analysis:\n{results['visual_analysis']['visual_summary']}\n\n")
        f.write(f"Final Summary:\n{results['final_summary']}\n")

    print("\n✅ Summary saved to 'video_summary.txt'")


🎬 VIDEO SUMMARIZATION PIPELINE

📹 Step 1: Extracting keyframes...
Video info: 2226 frames, 24.28 FPS, 91.68s duration
✅ Extracted 30 frames

🎤 Step 2: Processing audio...
⚠️ No audio track found in video

⚠️ No audio detected - proceeding with visual-only analysis

👁️ Step 3: Analyzing visual content...

🔍 Analyzing 30 frames...

✅ Visual analysis complete

📝 Step 4: Generating Default Summary...
📝 Generating Default Summary...
✅ Summary generated using Groq (LLaMA 3.3)

✅ PROCESSING COMPLETE

📊 FINAL SUMMARY
The video presents a series of visually striking images, set against a predominantly black background with intermittent flashes of green and white text. The opening scene features a black background with white text, setting the tone for a futuristic and technologically-driven narrative. As the video progresses, the viewer is introduced to a green screen with a black background, which serves as a canvas for various textual overlays. The text that appears on the screen provides insi

## 📊 Detailed Results Display


In [22]:
# Display detailed breakdown
if video_path and 'results' in locals():
    print("\n" + "=" * 60)
    print("📋 DETAILED BREAKDOWN")
    print("=" * 60)

    print("\n🎤 AUDIO TRANSCRIPTION:")
    print("-" * 60)
    print(results["audio_transcription"])

    print("\n\n👁️ VISUAL ANALYSIS:")
    print("-" * 60)
    for caption in results["visual_analysis"]["frame_captions"]:
        print(caption)

    if results["visual_analysis"]["ocr_texts"]:
        print("\n\n📝 EXTRACTED TEXT:")
        print("-" * 60)
        for text in set(results["visual_analysis"]["ocr_texts"]):
            print(f"  • {text}")

    print("\n\n📝 FINAL SUMMARY:")
    print("-" * 60)
    print(results["final_summary"])



📋 DETAILED BREAKDOWN

🎤 AUDIO TRANSCRIPTION:
------------------------------------------------------------
No audio track found in video.


👁️ VISUAL ANALYSIS:
------------------------------------------------------------
Frame 1: an image of a black background with white text
Frame 2: an image of a black background with a green light
Frame 3: the gps app on the iphone | On-screen text: 8*54 @ #, AGRON, 0, G, 8, Speed, Altitude, Battery, Spray, Progress, 5.0 mls, 20.0 m, Mission Type, Inspection, Start Mission
Frame 4: a green screen with a black background | On-screen text: 8*54 9 #, User Profile
Frame 5: a green screen with a black background and white text on it | On-screen text: 8*54 9 #, User Profile, Email: abdullah@gmail.com, veUaOSOEmlgDZrymogltVHSVkSw2, Account Created: 2025-11-27 13.58.29, UID:
Frame 6: the gps app on the iphone | On-screen text: 8*54 9, AGRON, G, 8, 0j9, Speed, Altitude, Battery, Spray, Progress, 5.0 mls, 20.0 m, Mission Type, Inspection, Start Mission
Frame 

## 🌐 API Server Setup

The API server allows your frontend to call the video summarizer. Run the server cell below to start the API.


In [23]:
# Initialize Flask app for API
app = Flask(__name__)
CORS(app)  # Enable CORS for frontend access

# Global variable to store processing status
processing_status = {"status": "idle", "message": ""}

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        "status": "healthy",
        "models_loaded": True,
        "llm_available": groq_client is not None or gemini_model is not None,
        "summary_styles": list(SUMMARY_STYLES.keys())
    })

@app.route('/api/video/upload', methods=['POST'])
def upload_video():
    """
    API endpoint to upload and process video.
    Expects multipart/form-data with 'file' field.
    """
    global processing_status

    try:
        if 'file' not in request.files:
            return jsonify({
                "status": "error",
                "message": "No file provided"
            }), 400

        file = request.files['file']

        if file.filename == '':
            return jsonify({
                "status": "error",
                "message": "No file selected"
            }), 400

        # Save uploaded file temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp_file:
            video_path = tmp_file.name
            file.save(video_path)

        # Update status
        processing_status = {"status": "processing", "message": "Processing video..."}

        # Process video
        num_frames = request.form.get('num_frames', 10, type=int)
        summary_style = request.form.get('summary_style', 'default', type=str)
        results = process_video(video_path, num_frames=num_frames, use_llm=True, summary_style=summary_style)

        # Cleanup
        if os.path.exists(video_path):
            os.remove(video_path)

        # Check for errors
        if "error" in results:
            processing_status = {"status": "error", "message": results["error"]}
            return jsonify({
                "status": "error",
                "message": results["error"]
            }), 500

        # Update status
        processing_status = {"status": "completed", "message": "Video processed successfully"}

        # Return results
        return jsonify({
            "status": "success",
            "summary": results["final_summary"],
            "audio_transcription": results.get("audio_transcription", ""),
            "visual_analysis": {
                "frame_count": len(results.get("visual_analysis", {}).get("frame_captions", [])),
                "ocr_texts": results.get("visual_analysis", {}).get("ocr_texts", [])
            }
        }), 200

    except Exception as e:
        processing_status = {"status": "error", "message": str(e)}
        return jsonify({
            "status": "error",
            "message": f"Error processing video: {str(e)}"
        }), 500

@app.route('/api/video/status', methods=['GET'])
def get_status():
    """Get current processing status"""
    return jsonify(processing_status)

@app.route('/api/video/process', methods=['POST'])
def process_video_api():
    """
    Alternative endpoint that accepts video as base64 or file path.
    """
    try:
        data = request.get_json()

        if not data:
            return jsonify({
                "status": "error",
                "message": "No data provided"
            }), 400

        # Handle base64 video (if provided)
        if 'video_base64' in data:
            video_data = base64.b64decode(data['video_base64'])
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp_file:
                tmp_file.write(video_data)
                video_path = tmp_file.name
        elif 'video_path' in data:
            video_path = data['video_path']
            if not os.path.exists(video_path):
                return jsonify({
                    "status": "error",
                    "message": "Video file not found"
                }), 404
        else:
            return jsonify({
                "status": "error",
                "message": "No video provided (need 'video_base64' or 'video_path')"
            }), 400

        # Process video
        num_frames = data.get('num_frames', 10)
        summary_style = data.get('summary_style', 'default')
        results = process_video(video_path, num_frames=num_frames, use_llm=True, summary_style=summary_style)

        # Cleanup if temporary file
        if 'video_base64' in data and os.path.exists(video_path):
            os.remove(video_path)

        if "error" in results:
            return jsonify({
                "status": "error",
                "message": results["error"]
            }), 500

        return jsonify({
            "status": "success",
            "summary": results["final_summary"],
            "audio_transcription": results.get("audio_transcription", ""),
            "visual_analysis": results.get("visual_analysis", {})
        }), 200

    except Exception as e:
        return jsonify({
            "status": "error",
            "message": f"Error: {str(e)}"
        }), 500

print("✅ API routes defined")
print("\nAvailable endpoints:")
print("  - POST /api/video/upload - Upload video file")
print("  - POST /api/video/process - Process video (JSON)")
print("  - GET /api/video/status - Get processing status")
print("  - GET /health - Health check")


✅ API routes defined

Available endpoints:
  - POST /api/video/upload - Upload video file
  - POST /api/video/process - Process video (JSON)
  - GET /api/video/status - Get processing status
  - GET /health - Health check


In [27]:
import time

# Start the Flask server
# For Colab, use ngrok or expose the port
# For local, the server will run on localhost:5000

global_public_url = None # Global variable to store the ngrok public URL

def run_server():
    global global_public_url
    # Check if running in Colab
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        # In Colab, use ngrok to expose the server
        print("Running in Colab - setting up ngrok...")
        !pip install -q pyngrok
        from pyngrok import ngrok

        # Start ngrok tunnel
        try:
            public_url = ngrok.connect(5000).public_url
            global_public_url = public_url # Store in global variable
            print(f"\n🌐 Public URL: {public_url}")
            print(f"📡 API Endpoint: {public_url}/api/video/upload")
            print("\n⚠️ Keep this cell running to keep the server active!")
        except Exception as e:
            print(f"⚠️ Error starting ngrok: {e}")
            print("Trying to run Flask locally without ngrok. This might not be accessible from other cells.")
            global_public_url = "http://localhost:5000"

    else:
        print("Running locally")
        global_public_url = "http://localhost:5000"
        print(f"Server will be available at: {global_public_url}")
        print(f"API Endpoint: {global_public_url}/api/video/upload")

    # Add a small delay to ensure server starts before returning
    time.sleep(2) # Wait for 2 seconds to ensure server is fully up

    # Run Flask app
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print("✅ API server started!")
print("\n📝 To stop the server, restart the runtime or interrupt the kernel.")

# Wait for the ngrok URL to be available before proceeding (if in Colab)
if 'IN_COLAB' in locals() and IN_COLAB:
    while global_public_url is None:
        time.sleep(1)
    print(f"Global public URL set: {global_public_url}")


Running in Colab - setting up ngrok...✅ API server started!

📝 To stop the server, restart the runtime or interrupt the kernel.



## 📡 API Usage Examples

### Example 1: Upload Video (Frontend Integration)

```javascript
// JavaScript/React example
const formData = new FormData();
formData.append('file', videoFile);

const response = await fetch('http://your-server-url/api/video/upload', {
  method: 'POST',
  body: formData
});

const data = await response.json();
console.log(data.summary); // Human-readable summary
```

### Example 2: Using cURL

```bash
curl -X POST http://localhost:5000/api/video/upload \
  -F "file=@your_video.mp4" \
  -F "num_frames=10"
```

### Example 3: Python Client

```python
import requests

with open('video.mp4', 'rb') as f:
    files = {'file': f}
    data = {'num_frames': 10}
    response = requests.post('http://localhost:5000/api/video/upload',
                           files=files, data=data)
    result = response.json()
    print(result['summary'])
```


In [32]:
import requests
import time

# Wait for the global_public_url to be set by the server thread
# This ensures the server has started and ngrok tunnel is up
while 'global_public_url' not in globals() or global_public_url is None:
    print("Waiting for API server to start...")
    time.sleep(2)

# Construct the full API endpoint using the global_public_url
api_endpoint = f"{global_public_url}/api/video/upload"

print(f"Attempting to connect to API at: {api_endpoint}")

with open('vedio1.mp4', 'rb') as f:
    files = {'file': f}
    data = {'num_frames': 40}
    try:
        response = requests.post(api_endpoint,
                               files=files, data=data)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        result = response.json()
        print(result['summary'])
    except requests.exceptions.ConnectionError as e:
        print(f"Connection Error: Could not connect to the API server. Please ensure the API server cell is running and ngrok tunnel is active. Error: {e}")
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: The server returned an error status. Error: {e}\nResponse: {response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


Attempting to connect to API at: http://localhost:5000/api/video/upload
🎬 VIDEO SUMMARIZATION PIPELINE

📹 Step 1: Extracting keyframes...
Video info: 2226 frames, 24.28 FPS, 91.68s duration
✅ Extracted 40 frames

🎤 Step 2: Processing audio...
⚠️ No audio track found in video

⚠️ No audio detected - proceeding with visual-only analysis

👁️ Step 3: Analyzing visual content...

🔍 Analyzing 40 frames...

✅ Visual analysis complete

📝 Step 4: Generating Default Summary...
📝 Generating Default Summary...


INFO:werkzeug:127.0.0.1 - - [30/Nov/2025 02:34:47] "POST /api/video/upload HTTP/1.1" 200 -


✅ Summary generated using Groq (LLaMA 3.3)

✅ PROCESSING COMPLETE
The video presents a series of images and on-screen text that appear to be related to a mission or navigation system. The visual content begins with a black background featuring white text, setting the tone for the rest of the video. As the video progresses, an iPhone is shown with a remote control button and a GPS app, suggesting that the mission may involve navigation or location-based tasks. The GPS app is displayed multiple times throughout the video, further emphasizing the importance of location and navigation in the mission.

The on-screen text provides additional context and details about the mission. The text mentions "Al Generated Mi" and "GIF, eastern field," which could be referring to a specific location or objective. The text also includes a date, "27, 2025," and an altitude reading of "50.0 m," which suggests that the mission may involve aerial or elevated operations. The phrase "Describe your mission in n

In [29]:
# Add your ngrok authtoken here. Replace 'YOUR_AUTHTOKEN' with your actual token.
# Get your authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken
!ngrok config add-authtoken YOUR_AUTHTOKEN

print("✅ ngrok authtoken configured. Please restart the runtime and re-run all cells.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ ngrok authtoken configured. Please restart the runtime and re-run all cells.


## 🔧 Advanced Configuration


In [ ]:
# Configuration options
CONFIG = {
    "num_frames": 10,  # Number of frames to extract (increase for longer videos)
    "whisper_model": "base",  # Options: tiny, base, small, medium (larger = better but slower)
    "blip_model": "base",  # Options: base, large (larger = better but slower)
    "summary_max_length": 200,  # Maximum length of summary
    "summary_min_length": 50,  # Minimum length of summary
    "ocr_confidence_threshold": 0.5,  # OCR confidence threshold (0-1)
    "enable_gpu": True,  # Use GPU if available
}

print("Current Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")


## 💡 Tips for Best Results

1. **Video Quality**: Higher quality videos produce better results
2. **Frame Count**: Increase `num_frames` for longer videos (10-20 frames recommended)
3. **Audio Quality**: Clear audio improves transcription accuracy
4. **Text Visibility**: Videos with clear on-screen text benefit from OCR
5. **Resource Management**: On Colab free tier, process shorter videos (< 5 minutes) for best performance

## 🚀 Performance Optimization

- Use smaller models (base/tiny) for faster processing
- Reduce `num_frames` for very long videos
- Process videos in chunks if they're very long
- Clear GPU cache between runs: `torch.cuda.empty_cache()`

## 📚 Model Information

This notebook uses industry-standard open-source models:

1. **Whisper (OpenAI)**: State-of-the-art speech recognition
   - Model: `openai/whisper-base`
   - Size: ~150MB
   - Best for: Audio transcription

2. **BLIP (Salesforce)**: Vision-language understanding
   - Model: `Salesforce/blip-image-captioning-base`
   - Size: ~990MB
   - Best for: Image captioning and visual understanding

3. **T5 (Google)**: Text-to-text transfer transformer
   - Model: `t5-small`
   - Size: ~240MB
   - Best for: Text summarization

4. **EasyOCR**: Optical Character Recognition
   - Free, open-source OCR
   - Best for: Extracting text from video frames

All models are free and can run on Colab free tier T4 GPU!
